In [15]:
using JuMP
using PiecewiseLinearOpt
using CPLEX
using DaChoppa
mip_solver = CplexSolver(CPX_PARAM_SCRIND=1, CPX_PARAM_EPINT=1e-9, 
    CPX_PARAM_EPRHS=1e-9, CPX_PARAM_EPGAP=1e-7);
global_solver = DaChoppaSolver(log_level=1, mip_solver=mip_solver);


In [16]:
# Problem definition
nctrl = 10; # number of control points
nt = 10; # number of time steps
r = 1; # radius of the motor
circ = 2*pi*r; # maximum circumference of the motor
circProfile = pi*r*ones(nt);
regrate = 0.05;

In [17]:
m = Model(solver = global_solver);

In [18]:
# Real-space variables
@variables m begin
    x[1:nctrl, 1:nt]
    absx[1:nctrl,1:nt]
    dx[1:nctrl, 1:nt]
    dx2[1:nctrl, 1:nt]
    y[1:nctrl, 1:nt]
    absy[1:nctrl,1:nt]
    dy[1:nctrl, 1:nt]   
    dy2[1:nctrl, 1:nt]
    nx[1:nctrl, 1:nt]
    ny[1:nctrl, 1:nt]
    l[1:nctrl, 1:nt]
    l2[1:nctrl, 1:nt]
    circ[1:nt]
    penalty[1:nt]
end

In [19]:
# Linear constraints
@constraints m begin
    l[:,:] .>= 0
    circ[:,:] .>= 0
    # Motion of flame front 
    x[1:nctrl,2:nt] .== x[1:nctrl,1:nt-1] + 
            0.5*nx[1:nctrl,1:nt-1]*regrate + 
            0.5*nx[1:nctrl,1:nt-1]*regrate
    y[1:nctrl,2:nt] .== y[1:nctrl,1:nt-1] + 
            0.5*ny[1:nctrl,1:nt-1]*regrate + 
            0.5*ny[1:nctrl,1:nt-1]*regrate
    dx[2:nctrl,1:nt] .== x[2:nctrl,1:nt] - x[1:nctrl-1,1:nt]
    dx[1,1:nt] .== x[1,1:nt] - x[nctrl,1:nt]
    dy[2:nctrl,1:nt] .== y[2:nctrl,1:nt] - y[1:nctrl-1,1:nt]
    dy[1,1:nt] .== y[1,1:nt] - y[nctrl,1:nt]
    l2[1:nctrl,1:nt] .== dx2[1:nctrl,1:nt] + dy2[1:nctrl,1:nt]
    absx[1:nctrl,1:nt] .>= x[1:nctrl,1:nt]
    absx[1:nctrl,1:nt] .>= -1*x[1:nctrl,1:nt]
    absy[1:nctrl,1:nt] .>= y[1:nctrl,1:nt]  
    absy[1:nctrl,1:nt] .>= -1*y[1:nctrl,1:nt]
    nx[1:nctrl,1:nt] .== -1*dy[1:nctrl,1:nt]
    ny[1:nctrl,1:nt] .== dx[1:nctrl,1:nt]
end
for i in 1:nt
    @constraint(m, circ[i] == sum(l[1:nctrl,i]))
    @constraint(m, penalty[i] >= circ[i] - circProfile[i])
    @constraint(m, penalty[i] >= -(circ[i] - circProfile[i]))
end

In [20]:
# Piecewise linearization
rn = r/10.
logabsx = piecewiselinear(m, absx, 0:rn:r, log.(0:rn:r))
logabsy = piecewiselinear(m, absy, 0:rn:r, log.(0:rn:r))

LoadError: [91mMethodError: no method matching piecewiselinear(::JuMP.Model, ::Array{JuMP.Variable,2}, ::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}, ::Array{Float64,1})[0m
Closest candidates are:
  piecewiselinear(::JuMP.Model, [91m::Union{JuMP.GenericAffExpr{Float64,JuMP.Variable}, JuMP.Variable}[39m, ::Any, ::Any; method) at C:\Users\Berk\.julia\v0.6\PiecewiseLinearOpt\src\jump.jl:24
  piecewiselinear(::JuMP.Model, [91m::Union{JuMP.GenericAffExpr{Float64,JuMP.Variable}, JuMP.Variable}[39m, ::Any, [91m::Function[39m; method) at C:\Users\Berk\.julia\v0.6\PiecewiseLinearOpt\src\jump.jl:19
  piecewiselinear(::JuMP.Model, [91m::JuMP.Variable[39m, [91m::Union{JuMP.GenericAffExpr{Float64,JuMP.Variable}, JuMP.Variable}[39m, ::Any, [91m::Any[39m, [91m::Function[39m; method) at C:\Users\Berk\.julia\v0.6\PiecewiseLinearOpt\src\jump.jl:516
  ...[39m

In [ ]:
# Purely convex (GP) constraints
for i in 1:nctrl
    for j in 1:nt
        @NLconstraint(m, exp(2*logabsx + 2*logabsy) <= exp(2*log(r)))
        @NLconstraint(m, exp(2*logabsx) == dx2[i,j])
        @NLconstraint(m, exp(2*logabsy) == dy2[i,j])
    end
end

In [ ]:
# # Piecewise linear constraints
# pldx2 = piecewiselinear(m, dx, 0:r/10.:r, u -> u^2)
# pldy2 = piecewiselinear(m, dy, 0:r/10.:r, u -> u^2)
# @constraint(m, dx2 == pldx2)
# @constraint(m, dy2 == pldy2)


In [14]:
@objective(m, Min, sum(penalty))

penalty[1] + penalty[2] + penalty[3] + penalty[4] + penalty[5]

In [ ]:
# m = soromotor_model(solver, nctrl, nt, r, regrate,circProfile)
solve(m)
#status = solve(m)